                                          10xResults v0.2

In [ ]:
import json
with open("config.json") as json_file:
    parameter = json.load(json_file)

# Load dataset

In [ ]:
import numpy as np
import pickle


with open(parameter["SAVE_DIR"]+"TCC_matrix.dat", 'rb') as f:
    T=pickle.load(f)
with open(parameter["SAVE_DIR"]+"pwise_dist_L1.dat", 'rb') as f:
    D_l1=pickle.load(f)
with open(parameter["SAVE_DIR"]+"nonzero_ec.dat", 'rb') as f:
    nonzero_ec=pickle.load(f)

ecfile_dir = parameter["kallisto"]["TCC_output"]+'matrix.ec'
eclist=np.loadtxt(ecfile_dir,dtype=str)    

from sklearn.preprocessing import normalize

TCC=T.T
T_norm = normalize(T, norm='l1', axis=0) 
T_normT = T_norm.transpose()
    
NUM_OF_CELLS=np.shape(T)[1]
print "NUM_OF_CELLS =", NUM_OF_CELLS
print "NUM_OF_nonzero_EC =", np.shape(T)[0]

In [ ]:
EC_dict = {}
for i in range(np.shape(eclist)[0]):
    EC_dict[i] = [int(x) for x in eclist[i,1].split(',')]
    
union=set()
for i in nonzero_ec:
    new = [tx for tx in EC_dict[i] if tx not in union] # filter out previously seen transcripts
    union.update(new) 
union_list=list(union) #union of all transctipt ids seen in nonzero eq.classes
NUM_OF_TX_inTCC = len(union)
print "NUM_OF_Transcripts =", NUM_OF_TX_inTCC #number of distinct transcripts in nonzero eq. classes 

### Inspect

In [ ]:
#sort eq. classes based on size
size_of_ec = [len(EC_dict[i]) for i in nonzero_ec]
ec_idx = [i[0] for i in sorted(enumerate(size_of_ec), key=lambda x:x[1])]
index_ec = np.array(ec_idx)

ec_sort_map={}
nonzero_ec_srt=[]; #init
for i in range(len(nonzero_ec)):
    nonzero_ec_srt+=[nonzero_ec[index_ec[i]]]
    ec_sort_map[nonzero_ec[index_ec[i]]] = i
nonzero_ec_srt = np.array(nonzero_ec_srt)

ec_size_sort=np.array(size_of_ec)[index_ec]
sumi = np.array(T.sum(axis=1))
sumi_sorted=sumi[index_ec]
total_num_of_umis = int(sumi_sorted.sum())
total_num_of_umis_per_cell = np.array(T.sum(axis=0))[0,:]

print "Total number of UMIs =", total_num_of_umis

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

fig, ax1 = plt.subplots()
ax1.plot(sorted(total_num_of_umis_per_cell)[::-1], 'b-',linewidth=2.0)
ax1.set_title('UMI counts per cell')
ax1.set_xlabel('cells (sorted by UMI counts)')
ax1.set_ylabel('UMI counts')
ax1.set_yscale("log", nonposy='clip')
# ax1.set_xscale("log", nonposy='clip')
ax1.grid(True)
ax1.grid(True,'minor')


fig, ax1 = plt.subplots()
ax1.plot(sorted(sumi.reshape(np.shape(sumi)[0]))[::-1], 'r-',linewidth=2.0)
ax1.set_title('UMI counts per eq. class')
ax1.set_xlabel('ECs (sorted by UMI counts)')
ax1.set_ylabel('UMI counts')
ax1.set_yscale("log", nonposy='clip')
# ax1.set_xscale("log", nonposy='clip')
ax1.grid(True)
ax1.grid(True,'minor')


cell_nonzeros=np.array(((T_norm!=0)).sum(axis=0))[0]

fig, ax1 = plt.subplots()
ax1.plot(total_num_of_umis_per_cell,cell_nonzeros, '.g',linewidth=2.0)
ax1.set_title('UMI counts vs nonzero ECs')
ax1.set_xlabel('total num of umis per cell')
ax1.set_ylabel('total num of nonzero ecs per cell')
ax1.set_yscale("log", nonposy='clip')
ax1.set_xscale("log", nonposy='clip')
ax1.grid(True)
ax1.grid(True,'minor')
plt.show()

In [ ]:
## TCC MEAN-VARIANCE 
def meanvar_plot(TCC_,alph=0.05):

    TCC_var=np.var(TCC_.todense(),axis=0)
    TCC_mean=np.mean(TCC_.todense(),axis=0)
    TCC_mean=np.array(TCC_mean)[0]
    TCC_var=np.array(TCC_var)[0]

    fig = plt.figure()
    N=TCC_.sum()
    C=TCC_.shape[0]
    ax = plt.gca()

    ax.plot(TCC_mean ,TCC_var,'.', c='blue', alpha=alph, markeredgecolor='none')

    xlims=[0.0001,10*TCC_mean.max()]
    ax.set_xlim(xlims)
    ax.set_ylim([0.0001,10*TCC_var.max()])
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.plot(xlims, [(C-1)*(xlims[0])**2, (C-1)*(xlims[1])**2], color='g', linestyle='-', linewidth=2)
    ax.plot(xlims, [(xlims[0]), (xlims[1])], color='k', linestyle='--', linewidth=1)
    ax.set_title("TCC Mean-Variance ["+str(TCC_.shape[1])+" TCCs in "+str(C)+" Cells]")
    ax.set_xlabel("mean(TCC)")
    ax.set_ylabel("var(TCC)")

In [ ]:
meanvar_plot(TCC,alph=0.5)

# Clustering

In [ ]:
from sklearn import cluster,manifold
import matplotlib.pyplot as plt
%matplotlib inline

def AffinityProp(D,pref,damp):
    aff= cluster.AffinityPropagation(affinity='precomputed',
                                     preference=pref,damping=damp, verbose=True)
    labels=aff.fit_predict(D)
    return labels

def spectral(k,D):
    spectral = cluster.SpectralClustering(n_clusters=k,affinity='precomputed')
    spectral.fit(D)
    labels = spectral.labels_
    return labels

def tSNE_pairwise(D):
    tsne = manifold.TSNE(n_components=2, random_state=213, metric='precomputed', n_iter=2000, verbose=1);
    X_tsne = tsne.fit_transform(D);
    return X_tsne

#### t-SNE

In [ ]:
X_tsne=tSNE_pairwise(D_l1)

In [ ]:
# Plot function with (nonrandom) colors corresponding to labels (star a specific label)
def stain_plot(X,labels,stain,title,nc=2,ax_lim=0,marksize=46):
    
    unique_labels = np.unique(labels)
    N = len(unique_labels)
    max_value = 16581375 #255**3
    interval = int(max_value / N)
    colors = [hex(I)[2:].zfill(6) for I in range(0, max_value, interval)]    
    color= [(int(i[:2], 16)/float(255), int(i[2:4], 16)/float(255), 
             int(i[4:], 16)/float(255)) for i in colors]
    i=0;
    plt.figure(figsize=(15,10))
    for label in unique_labels:
        ind = np.squeeze(labels == label)
        if label in stain: 
            plt.scatter(X[ind,0],X[ind,1],c='red',s=146,edgecolor='black',
                        lw = 0.5, alpha=1,marker='*',label=label)
        else:
            plt.scatter(X[ind,0],X[ind,1],c=color[i],s=marksize,edgecolor='lightgray',
                        lw = 0.5,label=label)        
        i+=1   
    plt.title(title)
    plt.gray()
    plt.legend(loc='upper right',bbox_to_anchor=(1.18, 1.01),ncol=nc)
    if ax_lim>0:
        plt.xlim([-ax_lim,ax_lim])  
        plt.ylim([-ax_lim,ax_lim])  
    plt.axis('off')
    plt.show()

In [ ]:
# Plot function with colors(heatmap) corresponding to the total number of umis per cell  (log10) 
def umi_counts_plot(X,total_num_of_umis_per_cell,title,ax_lim=0,,marksize=26):
    
    plt.figure(figsize=(15,10))
    plt.scatter(X_tsne[:,0],X_tsne[:,1],c=np.log10(total_num_of_umis_per_cell),s=marksize,edgecolor='black',lw=0.25,cmap='OrRd')
    plt.colorbar()
    plt.title(title)
    if ax_lim>0:
        plt.xlim([-ax_lim,ax_lim])  
        plt.ylim([-ax_lim,ax_lim])  
    plt.axis('off')

def color_plot(X,colorvec,title,ax_lim=0,marksize=26):
    
    plt.figure(figsize=(10,6))
    plt.scatter(X[:,0],X[:,1],c=np.log10(1+colorvec),s=marksize,edgecolor='black',lw=0.25,cmap='OrRd')
    plt.colorbar()
    plt.title(title)
    if ax_lim>0:
        plt.xlim([-ax_lim,ax_lim])  
        plt.ylim([-ax_lim,ax_lim])  
    plt.axis('off')
    plt.show()

#### Spectral Clustering

In [ ]:
num_of_clusters=2
similarity_mat=D_l1.max()-D_l1
labels_spectral = spectral(num_of_clusters,similarity_mat)

In [ ]:
stain_plot(X_tsne,labels_spectral,[],"TCC -- tSNE, spectral clustering")

#### Affinity Propagation

In [ ]:
pref = -np.median(D_l1)*np.ones(NUM_OF_CELLS)
labels_aff=AffinityProp(-D_l1,pref,0.5)
np.unique(labels_aff)

In [ ]:
stain_plot(X_tsne,labels_aff,[],"TCC -- tSNE, affinity propagation")

#### PCA

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(T_normT.todense())

In [ ]:
stain_plot(X_pca,labels_aff,[],"TCC -- PCA, affinity propagation")